In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from model import LinearRegressionModel

Load and preprocess data

In [ ]:
data = pd.read_csv("./weather.csv")
data.head()

In [ ]:
data = pd.get_dummies(data, columns=['Description'], drop_first=False, dtype=int)
data.head()

In [24]:
# Define features and target variable
X = data.drop(columns=['Temperature_c'])
Y = data['Temperature_c']

In [25]:
# Train-validation-test split
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X, Y, test_size=0.3, random_state=42)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=0.5, random_state=42)

In [26]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(data.corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation Matrix")
plt.show()

In [28]:
# Train Linear Regression Model without tuning
linear_model = LinearRegressionModel()
theta_before, _ = linear_model.fit(
    X_train_scaled, Y_train.to_numpy().reshape(-1, 1), learning_rate=1e-3, iterations=100
)
Y_pred_test_lr_before = linear_model.predict(X_test_scaled)

In [ ]:
# Tuning the Linear Regression Model
best_learning_rate, _ = linear_model.tune(
    X_train_scaled, Y_train.to_numpy().reshape(-1, 1),
    X_val_scaled, Y_val.to_numpy().reshape(-1, 1),
    [1e-7, 1e-6, 5e-5, 1e-5, 5e-4, 1e-4, 5e-3, 1e-3, 5e-2, 1e-2], iterations=1000
)

# Train Linear Regression Model with tuned learning rate
theta_after, _ = linear_model.fit(
    X_train_scaled, Y_train.to_numpy().reshape(-1, 1),
    learning_rate=best_learning_rate, iterations=1000
)
Y_pred_test_lr_after = linear_model.predict(X_test_scaled)

In [ ]:
# Calculate metrics for Linear Regression before tuning
mse_test_lr_before = mean_squared_error(Y_test, Y_pred_test_lr_before)
mae_test_lr_before = mean_absolute_error(Y_test, Y_pred_test_lr_before)
r2_test_lr_before = r2_score(Y_test, Y_pred_test_lr_before)

# Calculate metrics for Linear Regression after tuning
mse_test_lr_after = mean_squared_error(Y_test, Y_pred_test_lr_after)
mae_test_lr_after = mean_absolute_error(Y_test, Y_pred_test_lr_after)
r2_test_lr_after = r2_score(Y_test, Y_pred_test_lr_after)

# Print performance metrics for Linear Regression before tuning
print("Linear Regression Performance Metrics (Before Tuning):")
print(f"Test MSE: {mse_test_lr_before:.2f}")
print(f"Test MAE: {mae_test_lr_before:.2f}")
print(f"Test R²: {r2_test_lr_before:.2f}")


# Print performance metrics after tuning
print("\nLinear Regression Performance Metrics (After Tuning):")
print(f"Test MSE: {mse_test_lr_after:.2f}")
print(f"Test MAE: {mae_test_lr_after:.2f}")
print(f"Test R²: {r2_test_lr_after:.2f}")
